In [1]:
%load_ext autoreload
%autoreload 2

## API endpoints:


I. Endpoints for fetching rules

- Give me all the rules
- Give me rules for the datatypes X,Y,Z
- Give me rules applicable to the datatypes X,Y,Z given input filters

They return:
- a list of rules with their meta-data: id/name/desc (premise/query ?)

---------------------

II. Endpoinds for applying rules

- Apply rule A given input filters
- Apply rules A,B,C given input filters (?)

They return:
- a list of result resource IDs or full payloads (ids passed to ES views by atals?)


ISSUES TO RESOLVE:

- Convention for the filters: how atlas knows which params need to be specified for different rules
- Permission:
    - fetching rules: service account that can always access the project with rules.
    - applying rules: we need to get USER token from atlas and use it when applying the rules. handle access denied in some sessions.
- Service sessions for storing forge sessions for different users.
- Pagination of results.

In [2]:
import getpass

from kgforge.core import KnowledgeGraphForge

from inference_tools.utils import fetch_rules, check_premises, apply_rule, get_rule_parameters

In [3]:
ENDPOINT = "https://bbp.epfl.ch/nexus/v1"

In [4]:
TOKEN = getpass.getpass()

········


In [5]:
RULES_ORG = "bbp"
RULES_PROJECT = "inference-rules"
RULES_VIEW = "https://bbp.epfl.ch/neurosciencegraph/data/rule-view"

In [6]:
rules_forge = KnowledgeGraphForge(
    "../../configs/new-forge-config.yaml",
    endpoint=ENDPOINT,
    token=TOKEN, 
    bucket=f"{RULES_ORG}/{RULES_PROJECT}")

In [7]:
# This could be replaced by any util kg-inference-api implements to handle user forge sessions
FORGE_SESSIONS = {}
def allocate_forge_session(org, project):
    if (org, project) not in FORGE_SESSIONS:
        session = KnowledgeGraphForge(
            "../../configs/new-forge-config.yaml",
            endpoint=ENDPOINT,
            token=TOKEN, 
            bucket=f"{org}/{project}")
        FORGE_SESSIONS[(org, project)] = session
    return FORGE_SESSIONS[(org, project)]

# Fetch rules

1. Fetch all the rules

In [9]:
rules = fetch_rules(rules_forge, RULES_VIEW)
for r in rules:
    print("Name: ", r["name"])
    print("Desc: ", r["description"])
    print("Target resource type: ", r["targetResourceType"])
    params = get_rule_parameters(r)
    print("Input parameters: ")
    for name, payload in params.items():
        print("\t", name)
        print("\t\t", payload)
    print()

Name:  All-aspect morphology recommendation
Desc:  Get closest morphologies according to their location and shape
Target resource type:  NeuronMorphology
Input parameters: 
	 TargetResourceParameter
		 {'@type': 'uri', 'description': 'URI of the similarity search target entity.', 'name': 'TargetResourceParameter'}
	 IgnoreModelsParameter
		 {'@type': 'list', 'description': 'List of URIs of similarity models to ignore.', 'name': 'IgnoreModelsParameter', 'optional': True}

Name:  Location-based morphology recommendation
Desc:  Get closest morphologies according to their location given by: coordinates, brain region embedding (ontology-based), axon/dendrite co-projection.
Target resource type:  NeuronMorphology
Input parameters: 
	 TargetResourceParameter
		 {'@type': 'uri', 'description': 'URI of the similarity search target entity.', 'name': 'TargetResourceParameter'}
	 IgnoreModelsParameter
		 {'@type': 'list', 'description': 'List of URIs of similarity models to ignore.', 'name': 'Igno

2. Fetch rules for a particular data type

In [10]:
morph_rules = fetch_rules(rules_forge, RULES_VIEW, resource_types=["NeuronMorphology"])
for r in morph_rules:
#     print(r)
    print("Name: ", r["name"])
    print("Desc: ", r["description"])
    print("Target resource type: ", r["targetResourceType"])
    params = get_rule_parameters(r)
    print("Input parameters: ")
    for name, payload in params.items():
        print("\t", name)
        print("\t\t", payload)
    print()

Name:  All-aspect morphology recommendation
Desc:  Get closest morphologies according to their location and shape
Target resource type:  NeuronMorphology
Input parameters: 
	 TargetResourceParameter
		 {'@type': 'uri', 'description': 'URI of the similarity search target entity.', 'name': 'TargetResourceParameter'}
	 IgnoreModelsParameter
		 {'@type': 'list', 'description': 'List of URIs of similarity models to ignore.', 'name': 'IgnoreModelsParameter', 'optional': True}

Name:  Location-based morphology recommendation
Desc:  Get closest morphologies according to their location given by: coordinates, brain region embedding (ontology-based), axon/dendrite co-projection.
Target resource type:  NeuronMorphology
Input parameters: 
	 TargetResourceParameter
		 {'@type': 'uri', 'description': 'URI of the similarity search target entity.', 'name': 'TargetResourceParameter'}
	 IgnoreModelsParameter
		 {'@type': 'list', 'description': 'List of URIs of similarity models to ignore.', 'name': 'Igno

# Check premises of rules (find rules applicable to input filters)

In [18]:
input_filters = {
    "TargetResourceParameter": "https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/85a140b6-8495-4838-9e4f-e347b2a7838f"
}

If premise is satified given the input filters, the rule is applicable.

In [19]:
for rule in morph_rules:
    print(rule["name"])
    print("\t-> Input filters:")
    print("\t\tPremise satisfied: ", check_premises(allocate_forge_session, rule, input_filters))

All-aspect morphology recommendation
	-> Input filters:
		Premise satisfied:  True
Location-based morphology recommendation
	-> Input filters:
		Premise satisfied:  True
Shape-based morphology recommendation
	-> Input filters:
		Premise satisfied:  True


# Excecute inference

Rule application returns a collection of resources to display in the atlas

By default the premise is checked again when the rule is applied

In [20]:
for rule in morph_rules:
    print(rule["name"])
    print("\t Results:")
    results = apply_rule(allocate_forge_session, rule, input_filters)
    for r in results:
        print("\t\t", r)
    print()

All-aspect morphology recommendation
	 Results:
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/5c5d6a1a-9102-445a-8d37-aa4e6f8acb88'}
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/2d25225f-cdf5-4c55-9f3c-b4f4c71ed1e5'}
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/15c8a4c5-807c-4512-bc03-15b7ff987831'}
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/d96779ca-fa70-490b-af1a-de1f8cc21d3b'}
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/e7634847-3903-4d18-a94e-fc8fb2be3f7c'}
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/9457c4e0-a4c5-4f3e-b535-bcb4874b53c6'}
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/518db742-56e6-433d-8201-a1754390636c'}
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphologies/cf191ee6-3d54-4e5f-9444-3d8167d626ed'}
		 {'id': 'https://bbp.epfl.ch/neurosciencegraph/data/ne